In [1]:
import pandas as pd
import numpy as np
import random
from collections import Counter
import matplotlib.pyplot as plt
import time
import datetime

from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.grid_search import GridSearchCV
import sklearn.preprocessing
import sklearn.cross_validation
import xgboost as xgb

%matplotlib inline

In [2]:
train = pd.read_csv('./train.csv').set_index("ID")
test = pd.read_csv('./test.csv').set_index("ID")

/home/ubuntu/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2871: DtypeWarning: Columns (8,9,10,11,12,43,157,196,214,225,228,229,231,235,238) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/ubuntu/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2871: DtypeWarning: Columns (8,9,10,11,12,43,157,167,177,196,214,225,228,229,231,235,238) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
drop_cols = [c for c in train.columns if len(np.unique(train[c])) == 1]#Should later drop columns from train and test
#print drop_cols
train.drop(drop_cols, axis = 1, inplace = True)
test.drop(drop_cols, axis = 1, inplace = True)

/home/ubuntu/anaconda/lib/python2.7/site-packages/numpy/lib/arraysetops.py:200: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  flag = np.concatenate(([True], aux[1:] != aux[:-1]))


In [4]:
object_cols = train.columns[train.dtypes == 'object']
dates_cols =['VAR_0073', 'VAR_0075', 'VAR_0156', 'VAR_0157', 'VAR_0158', 'VAR_0159', 'VAR_0166', 'VAR_0167', 'VAR_0168', 'VAR_0169', 'VAR_0176', 'VAR_0177', 'VAR_0178','VAR_0179', 'VAR_0204', 'VAR_0217']

In [5]:
redundant_0008 = ['VAR_0008', 'VAR_0009', 'VAR_0010','VAR_0011','VAR_0012', 'VAR_0043','VAR_0044','VAR_0196', 'VAR_0229', ] #Contains redundant information
train.drop(redundant_0008, axis = 1, inplace = True)
test.drop(redundant_0008, axis = 1, inplace = True)

In [8]:
train['NaN'] = 0
test['NaN'] = 0

In [10]:
for c in test.columns: #Engineered Feature #1
    train['NaN'] += pd.isnull(train[c])
    test['NaN'] += pd.isnull(test[c])
    

In [6]:
def to_date(date_str):
    if pd.notnull(date_str):
        return datetime.datetime.strptime(date_str, '%d%b%y:%H:%M:%S')
    else:
        return date_str

In [12]:
for c in dates_cols:
    train[c] = train[c].apply(to_date)
    test[c] = test[c].apply(to_date)

In [13]:
diff = list(set(test.columns[test.dtypes == 'datetime64[ns]']) - set(train.columns[train.dtypes == 'datetime64[ns]'])) ##test got converted into date time format while train is still object format

In [16]:
train.drop(diff, axis = 1, inplace = True)
test.drop(diff, axis = 1, inplace = True)

In [17]:
dates_cols_fixed = train.columns[train.dtypes == 'datetime64[ns]']

In [18]:
dates_cols_fixed

Index([u'VAR_0073', u'VAR_0075', u'VAR_0204', u'VAR_0217'], dtype='object')

In [80]:
train['VAR_0204-VAR_0217'] = (train['VAR_0204'] - train['VAR_0217']).apply(lambda x: x.astype('timedelta64[D]')/np.timedelta64(1, 'D'))
test['VAR_0204-VAR_0217'] = (test['VAR_0204'] - test['VAR_0217']).apply(lambda x: x.astype('timedelta64[D]')/np.timedelta64(1, 'D'))

In [85]:
train['VAR_0204-VAR_0075'] = (train['VAR_0204'] - train['VAR_0075']).apply(lambda x: x.astype('timedelta64[D]')/np.timedelta64(1, 'D'))
test['VAR_0204-VAR_0075'] = (test['VAR_0204'] - test['VAR_0075']).apply(lambda x: x.astype('timedelta64[D]')/np.timedelta64(1, 'D'))

In [91]:
train['VAR_0073_ISNULL'] = pd.isnull(train['VAR_0073']).astype('int')
test['VAR_0073_ISNULL'] = pd.isnull(test['VAR_0073']).astype('int')

In [95]:
errors_dates = [] #Does not matter because will convert to float
for c in dates_cols_fixed: 
    try:
        train[c+'_YEAR'] = train[c].apply(lambda x: x.year)
        test[c+'_YEAR'] = test[c].apply(lambda x: x.year)
        train[c+'_MONTH'] = train[c].apply(lambda x: x.month)
        test[c+'_MONTH'] = test[c].apply(lambda x: x.month)
    except:
        errors.append(c)

In [100]:
train.drop(dates_cols_fixed, axis = 1, inplace=True)
test.drop(dates_cols_fixed, axis = 1, inplace=True)

In [103]:
object_cols = train.columns[train.dtypes == 'object']

In [106]:
encoders = {}
errors = []
for col in object_cols:
    if col not in dates_cols:
        encoders[col] = LabelEncoder()
        encoders[col].fit(list(train[col]) + list(test[col]))
        try:
            train[col] = encoders[col].transform(train[col])
            test[col] = encoders[col].transform(test[col])
        except:
            errors.append(col)
            #del train[col]
            #del test[col]


In [112]:
object_cols = train.columns[train.dtypes == 'object']


In [118]:
train.drop(object_cols, axis = 1, inplace = True)
test.drop(object_cols, axis = 1, inplace = True)

In [53]:
X = train.drop('target', axis = 1).values
y = train.target.values
X_test = test.values


In [54]:
imputer = sklearn.preprocessing.Imputer(strategy='median')
X = imputer.fit_transform(X)
X_test = imputer.transform(X_test)

In [119]:
#rf = RandomForestClassifier(n_estimators = 45)
#sklearn.cross_validation.cross_val_score(rf, X, y, scoring = 'roc_auc')
#rf.fit(X,y)


In [38]:
#y_pred = rf.predict(X_test)

In [39]:
# MAKING SUBMISSION
#submission = pd.DataFrame(y_pred, index=test.index, columns=['target'])
#submission.index.name = 'ID'
#submission.to_csv('B_RF1.csv')

In [51]:
#xgb_clf = xgb.XGBClassifier(max_depth=8, learning_rate=.005, n_estimators=800, subsample=.8, \
#                            colsample_bytree=.8)

In [46]:
#sklearn.cross_validation.cross_val_score(xgb_clf, X, y)

array([ 0.78783747,  0.78810601,  0.79041087])

In [44]:
#y_pred_xgb = xgb_clf.predict(X_test)

In [43]:
# MAKING SUBMISSION
#submission = pd.DataFrame(y_pred_xgb, index=test.index, columns=['target'])
#submission.index.name = 'ID'
#submission.to_csv('B_XGB1.csv')

In [68]:
X_train, X_val, y_train, y_val = sklearn.cross_validation.train_test_split(X, y, random_state = 42)

In [71]:
xgb_clf = xgb.XGBClassifier(max_depth=8, learning_rate=.006, n_estimators=2000, subsample=.8, \
                            colsample_bytree=.8)

In [72]:
xgb_clf.fit(X_train, y_train, eval_set = [(X_val, y_val)], eval_metric='auc', early_stopping_rounds=20)

Will train until validation_0 error hasn't decreased in 10 rounds.
[0]	validation_0-auc:0.714568
[1]	validation_0-auc:0.734646
[2]	validation_0-auc:0.740802
[3]	validation_0-auc:0.743748
[4]	validation_0-auc:0.746256
[5]	validation_0-auc:0.748371
[6]	validation_0-auc:0.749066
[7]	validation_0-auc:0.749804
[8]	validation_0-auc:0.750479
[9]	validation_0-auc:0.750624
[10]	validation_0-auc:0.751425
[11]	validation_0-auc:0.751871
[12]	validation_0-auc:0.752167
[13]	validation_0-auc:0.753314
[14]	validation_0-auc:0.753688
[15]	validation_0-auc:0.754137
[16]	validation_0-auc:0.754606
[17]	validation_0-auc:0.754933
[18]	validation_0-auc:0.755113
[19]	validation_0-auc:0.755371
[20]	validation_0-auc:0.755499
[21]	validation_0-auc:0.755536
[22]	validation_0-auc:0.755538
[23]	validation_0-auc:0.755578
[24]	validation_0-auc:0.755956
[25]	validation_0-auc:0.756347
[26]	validation_0-auc:0.756604
[27]	validation_0-auc:0.756763
[28]	validation_0-auc:0.756758
[29]	validation_0-auc:0.756758
[30]	validati

XGBClassifier(base_score=0.5, colsample_bytree=0.8, gamma=0,
       learning_rate=0.005, max_delta_step=0, max_depth=8,
       min_child_weight=1, missing=None, n_estimators=5000, nthread=-1,
       objective='binary:logistic', seed=0, silent=True, subsample=0.8)

In [73]:
y_pred_xgb = xgb_clf.predict(X_test)

In [75]:


# MAKING SUBMISSION
submission = pd.DataFrame(y_pred_xgb, index=test.index, columns=['target'])
submission.index.name = 'ID'
submission.to_csv('B_XGB7919.csv')

In [ ]:
xgb_clf2 = xgb.XGBClassifier(max_depth=8, learning_rate=.005, n_estimators=3000, subsample=.8, \
                            colsample_bytree=.8)

xgb_clf2.fit(X, y)
y_pred_xgb2 = xgb_clf.predict(X_test)

# MAKING SUBMISSION
submission = pd.DataFrame(y_pred_xgb2, index=test.index, columns=['target'])
submission.index.name = 'ID'
submission.to_csv('B_XGB3.csv')